In [12]:
pip install arch

   ---------------------------------------- 0.0/924.9 kB ? eta -:--:--
   ---------------------------------------- 10.2/924.9 kB ? eta -:--:--
   - ------------------------------------- 41.0/924.9 kB 495.5 kB/s eta 0:00:02
   --- ----------------------------------- 81.9/924.9 kB 657.6 kB/s eta 0:00:02
   ----- -------------------------------- 143.4/924.9 kB 950.9 kB/s eta 0:00:01
   -------- ------------------------------- 204.8/924.9 kB 1.0 MB/s eta 0:00:01
   ---------- ----------------------------- 245.8/924.9 kB 1.1 MB/s eta 0:00:01
   ------------ --------------------------- 297.0/924.9 kB 1.1 MB/s eta 0:00:01
   ---------------- ----------------------- 378.9/924.9 kB 1.2 MB/s eta 0:00:01
   ----------------- ---------------------- 409.6/924.9 kB 1.1 MB/s eta 0:00:01
   --------------------- ------------------ 491.5/924.9 kB 1.2 MB/s eta 0:00:01
   ------------------------ --------------- 573.4/924.9 kB 1.3 MB/s eta 0:00:01
   --------------------------- ------------ 624.6/924.9 k

In [13]:
import pandas as pd
from arch import arch_model

# Load the data
data = pd.read_csv('C:/Users/HP/Desktop/ROHAN A6/Amazon.csv')
data['Date'] = pd.to_datetime(data['Date'])
data = data.sort_values('Date')

# Calculate daily returns
data['Returns'] = data['Price'].pct_change() * 100
data.dropna(inplace=True)

# Fit a GARCH(1,1) model
garch_model = arch_model(data['Returns'], vol='Garch', p=1, q=1)
garch_fitted = garch_model.fit()

# Print model summary
print(garch_fitted.summary())

# Forecasting the next 3 months (63 trading days)
forecast_horizon = 63
forecast = garch_fitted.forecast(horizon=forecast_horizon)
forecast_variance = forecast.variance.values[-1]
forecast_std_dev = np.sqrt(forecast_variance).mean()
print(f"The forecasted volatility for the next 3 months is: {forecast_std_dev:.2f}%")


Iteration:      1,   Func. Count:      6,   Neg. LLF: 5935.3237342358825
Iteration:      2,   Func. Count:     14,   Neg. LLF: 788622.293452479
Iteration:      3,   Func. Count:     21,   Neg. LLF: 2512.7367952655063
Iteration:      4,   Func. Count:     28,   Neg. LLF: 2451.568058035947
Iteration:      5,   Func. Count:     34,   Neg. LLF: 2446.8960206633055
Iteration:      6,   Func. Count:     40,   Neg. LLF: 2448.7674045582826
Iteration:      7,   Func. Count:     46,   Neg. LLF: 2444.176340189496
Iteration:      8,   Func. Count:     51,   Neg. LLF: 2444.1710024546237
Iteration:      9,   Func. Count:     56,   Neg. LLF: 2444.1709321399494
Iteration:     10,   Func. Count:     61,   Neg. LLF: 2444.1709303063717
Iteration:     11,   Func. Count:     66,   Neg. LLF: 2444.170929535647
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2444.170929535647
            Iterations: 11
            Function evaluations: 66
            Gradient evaluatio

In [14]:
import numpy as np
from statsmodels.tsa.stattools import adfuller
from arch import arch_model

# Convert the Date column to datetime
data['Date'] = pd.to_datetime(data['Date'])

# Sort the data by Date
data = data.sort_values('Date')

# Calculate daily returns as the percentage change in Price
data['Returns'] = data['Price'].pct_change() * 100

# Drop missing values
data.dropna(inplace=True)

# Check the first few rows with returns calculated
data.head()


,Date,Price,Open,High,Low,Vol.,Change %,Returns
1119,2020-02-05,101.99,103.55,103.55,101.60,87.52M,-0.48%,-0.478142
1118,2020-02-06,102.51,102.05,102.81,101.24,63.66M,0.51%,0.509854
1117,2020-02-07,103.96,102.10,104.93,101.91,101.91M,1.41%,1.414496
1116,2020-02-10,106.70,104.25,106.78,104.25,101.12M,2.64%,2.635629
1115,2020-02-11,107.54,107.55,109.30,106.80,114.92M,0.79%,0.787254


In [15]:
from statsmodels.stats.diagnostic import het_arch
import statsmodels.api as sm

# Perform the Lagrange Multiplier (LM) test for ARCH effects
returns = data['Returns']
test_result = het_arch(returns.dropna())

test_result_summary = {
    'LM Statistic': test_result[0],
    'p-value': test_result[1],
    'F-statistic': test_result[2],
    'F p-value': test_result[3]
}

test_result_summary


{'LM Statistic': 48.79269667929363,
 'p-value': 4.4454449613910413e-07,
 'F-statistic': 5.0530347353195975,
 'F p-value': 3.1853678399519417e-07}

In [16]:
# Check the column names to ensure 'Returns' was correctly added
data.columns


Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Change %', 'Returns'], dtype='object')

In [17]:
# Recalculate daily returns as the percentage change in Price
data['Returns'] = data['Price'].pct_change() * 100

# Drop missing values that arise from the percentage change calculation
data.dropna(inplace=True)

# Check if 'Returns' column is now present
data.head()


,Date,Price,Open,High,Low,Vol.,Change %,Returns
1118,2020-02-06,102.51,102.05,102.81,101.24,63.66M,0.51%,0.509854
1117,2020-02-07,103.96,102.10,104.93,101.91,101.91M,1.41%,1.414496
1116,2020-02-10,106.70,104.25,106.78,104.25,101.12M,2.64%,2.635629
1115,2020-02-11,107.54,107.55,109.30,106.80,114.92M,0.79%,0.787254
1114,2020-02-12,108.00,108.16,109.01,107.76,66.69M,0.43%,0.427748


In [18]:
# Perform the Lagrange Multiplier (LM) test for ARCH effects
returns = data['Returns']
test_result = het_arch(returns.dropna())

test_result_summary = {
    'LM Statistic': test_result[0],
    'p-value': test_result[1],
    'F-statistic': test_result[2],
    'F p-value': test_result[3]
}

test_result_summary


{'LM Statistic': 48.664102537477135,
 'p-value': 4.6930746903142756e-07,
 'F-statistic': 5.039269604473466,
 'F p-value': 3.3694698913189e-07}

In [19]:
from statsmodels.tsa.api import GARCH

# Fit a GARCH(1,1) model
model = sm.tsa.GARCH(returns, order=(1, 1))
fitted_model = model.fit()

# Summarize the model
summary = fitted_model.summary()

# Forecast volatility for the next three months (approximately 63 trading days)
forecasts = fitted_model.forecast(steps=63)
forecast_variance = forecasts.variance.values[-1]
forecast_std = np.sqrt(forecast_variance)  # Convert variance to standard deviation (volatility)

summary, forecast_std


ImportError: cannot import name 'GARCH' from 'statsmodels.tsa.api' (D:\anaconda\Lib\site-packages\statsmodels\tsa\api.py)

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# Fit a simple ARIMA model for mean specification (mean equation)
mean_model = ARIMA(returns, order=(1, 0, 0))
mean_results = mean_model.fit()

# Extract residuals
residuals = mean_results.resid

# Fit the GARCH model on residuals
garch_model = sm.tsa.ARIMA(residuals**2, order=(1, 0, 1))
garch_results = garch_model.fit()

# Summarize the GARCH model
garch_summary = garch_results.summary()

# Forecast the next three months of volatility (approximately 63 trading days)
forecast_horizon = 63
volatility_forecast = np.sqrt(garch_results.forecast(steps=forecast_horizon).variance[-1])

garch_summary, volatility_forecast


In [ ]:
from statsmodels.tsa.arima.model import ARIMA
import numpy as np

# Fit a simple ARIMA model for mean specification (mean equation)
mean_model = ARIMA(returns, order=(1, 0, 0))
mean_results = mean_model.fit()

# Extract residuals
residuals = mean_results.resid

# Fit the GARCH model on residuals (use squared residuals to model the variance)
garch_model = sm.tsa.ARIMA(residuals**2, order=(1, 0, 1))
garch_results = garch_model.fit()

# Forecast the next three months of volatility (approximately 63 trading days)
forecast_horizon = 63
forecast = garch_results.get_forecast(steps=forecast_horizon)
forecast_variance = forecast.predicted_mean
forecast_std_dev = np.sqrt(forecast_variance).mean()  # Take mean to get average forecasted volatility

forecast_std_dev
